# 地理空間データの加工・可視化
- 地理空間データは、多くの市場で使用可能なデータとなっている。
    - 公的機関が一般公開していてとても有用
    - 様々なデータを組み合わせることが可能な応用力の高いデータ
## データ分析においては
- 商圏分析
- 適地選定
- 顧客分析
- BCP（事業継続計画: 防災・災害対策）
- 掃除ロボットの制御、紛失防止タグのような屋内測位技術

に用いられる。

## AI 予測であれば
- 感染症の感染予測マップ
- 不動産売買の適正価格予測

などに活用。